В формулировке заданий будет использоваться понятие **worker**. Это слово обозначает какую-то единицу параллельного выполнения, в случае питона это может быть **поток** или **процесс**, выбирайте то, что лучше будет подходить к конкретной задаче

# Задание 1 (7 баллов)

В одном из заданий по ML от вас требовалось написать кастомную реализацию Random Forest. Её проблема состоит в том, что она работает медленно, так как использует всего один поток для работы. Добавление параллельного программирования в код позволит получить существенный прирост в скорости обучения и предсказаний.

В данном задании от вас требуется добавить возможность обучать случайный лес параллельно и использовать параллелизм для предсказаний. Для этого вам понадобится:
1. Добавить аргумент `n_jobs` в метод `fit`. `n_jobs` показывает количество worker'ов, используемых для распараллеливания
2. Добавить аргумент `n_jobs` в методы `predict` и `predict_proba`
3. Реализовать функционал по распараллеливанию в данных методах

В результате код `random_forest.fit(X, y, n_jobs=2)` и `random_forest.predict(X, y, n_jobs=2)` должен работать в ~1.5-2 раза быстрее, чем `random_forest.fit(X, y, n_jobs=1)` и `random_forest.predict(X, y, n_jobs=1)` соответственно

Если у вас по каким-то причинам нет кода случайного леса из ДЗ по ML, то вы можете написать его заново или попросить у однокурсника. *Детали* реализации ML части оцениваться не будут, НО, если вы поломаете логику работы алгоритма во время реализации параллелизма, то за это будут сниматься баллы

В задании можно использовать только модули из **стандартной библиотеки** питона, а также функции и классы из **sklearn** при помощи которых вы изначально писали лес

In [1]:
!pip install sklearn.utils


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [49]:
from joblib import Parallel, delayed
SEED = 111
random.seed(SEED)
np.random.seed(SEED)
class RandomForestClassifierCustom(BaseEstimator):
    def __init__(
        self, n_estimators=10, max_depth=None, max_features=None, random_state=SEED
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feat_ids_by_tree = []

    def fit(self, X, y, n_jobs=1):
        self.classes_ = sorted(np.unique(y))

        pseudoselection = np.random.choice((X.shape[0]), size=X.shape[0], replace=True) # Создайте псевдовыборку

        def fit_tree(i):
            np.random.seed(self.random_state + i) # Зафиксируйте генератор случайных чисел
            feat_ids = np.random.choice(X.shape[1], size=self.max_features, replace=False) # Выберите max_features признаков без возвращения
            self.feat_ids_by_tree.append(feat_ids) # Сохраните список выбранных признаков
            classifier = DecisionTreeClassifier(max_depth=self.max_depth,
                                                max_features = self.max_features,
                                                random_state=self.random_state + i)
            classifier.fit(X[pseudoselection][:, feat_ids], y[pseudoselection])
            return classifier

        self.trees = Parallel(n_jobs=n_jobs)(delayed(fit_tree)(i) for i in range(self.n_estimators))

        return self

    def predict_proba(self, X, n_jobs=1):
        def predict_proba_tree(tree, feature):
            return tree.predict_proba(X[:, feature])
        #np.random.seed(random_state)


        y_pred = np.zeros((X.shape[0], len(self.classes_)))
        y_trees = Parallel(n_jobs=n_jobs)(delayed(predict_proba_tree)(tree, feature) for tree, feature in zip(self.trees, self.feat_ids_by_tree))
        for y_tree in y_trees:
            y_pred += y_tree

        return y_pred / self.n_estimators

    def predict(self, X, n_jobs=1):
        probas = self.predict_proba(X, n_jobs=n_jobs)
        predictions = np.argmax(probas, axis=1)

        return predictions
X, y = make_classification(n_samples=100000)

In [52]:
from joblib import Parallel, delayed

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(
        self, n_estimators=10, max_depth=None, max_features=None, random_state=SEED
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feat_ids_by_tree = []

    def fit(self, X, y, n_jobs=1):
        self.classes_ = sorted(np.unique(y))

        def fit_tree(i):
            np.random.seed(self.random_state + i) # Зафиксируйте генератор случайных чисел
            feat_ids = np.random.choice(X.shape[1], size=self.max_features, replace=False) # Выберите max_features признаков без возвращения
            self.feat_ids_by_tree.append(feat_ids) # Сохраните список выбранных признаков
            pseudoselection = np.random.choice((X.shape[0]), size=X.shape[0], replace=True) # Создайте псевдовыборку
            classifier = DecisionTreeClassifier(max_depth=self.max_depth,
                                                max_features = self.max_features,
                                                random_state=self.random_state + i)
            classifier.fit(X[pseudoselection][:, feat_ids], y[pseudoselection])
            return classifier

        # Set the random seed the same way in both runs
        np.random.seed(self.random_state)

        self.trees = Parallel(n_jobs=n_jobs)(delayed(fit_tree)(i) for i in range(self.n_estimators))

        return self

    def predict_proba(self, X, n_jobs=1):
        def predict_proba_tree(tree, feature):
            return tree.predict_proba(X[:, feature])

        y_pred = np.zeros((X.shape[0], len(self.classes_)))

        # Use the same order of trees and feature indices in both runs
        y_trees = Parallel(n_jobs=n_jobs)(delayed(predict_proba_tree)(tree, feature) for tree, feature in zip(self.trees, self.feat_ids_by_tree))
        
        for y_tree in y_trees:
            y_pred += y_tree

        return y_pred / self.n_estimators

    def predict(self, X, n_jobs=1):
        probas = self.predict_proba(X, n_jobs=n_jobs)
        predictions = np.argmax(probas, axis=1)

        return predictions

X, y = make_classification(n_samples=100000)

In [84]:
from joblib import Parallel, delayed

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(
        self, n_estimators=10, max_depth=None, max_features=None, random_state=SEED
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feat_ids_by_tree = []

    def fit(self, X, y, n_jobs=1):
        self.classes_ = sorted(np.unique(y))

        def fit_tree(i):
            np.random.seed(self.random_state + i) # Зафиксируйте генератор случайных чисел
            feat_ids = np.random.choice(X.shape[1], size=self.max_features, replace=False) # Выберите max_features признаков без возвращения
            self.feat_ids_by_tree.append(feat_ids) # Сохраните список выбранных признаков
            pseudoselection = np.random.choice((X.shape[0]), size=X.shape[0], replace=True) # Создайте псевдовыборку
            classifier = DecisionTreeClassifier(max_depth=self.max_depth,
                                                max_features = self.max_features,
                                                random_state=self.random_state + i)
            classifier.fit(X[pseudoselection][:, feat_ids], y[pseudoselection])
            return classifier

        self.trees = Parallel(n_jobs=n_jobs)(delayed(fit_tree)(i) for i in range(self.n_estimators))

        # sort the trees and feature indices based on the random seed
        if self.trees and self.feat_ids_by_tree:
            indices = np.argsort([self.random_state + sum(x[1]) for x in zip(self.trees, self.feat_ids_by_tree)], kind='mergesort')
            self.trees = [self.trees[i] for i in indices]
            self.feat_ids_by_tree = [self.feat_ids_by_tree[i] for i in indices]

        return self
    def predict_proba(self, X, n_jobs=1):
        def predict_proba_tree(tree, feature):
            return tree.predict_proba(X[:, feature])

        y_pred = np.zeros((X.shape[0], len(self.classes_)))
        y_trees = Parallel(n_jobs=n_jobs)(delayed(predict_proba_tree)(tree, feature) for tree, feature in zip(self.trees, self.feat_ids_by_tree))
        for y_tree in y_trees:
            y_pred += y_tree

        return y_pred / self.n_estimators

    def predict(self, X, n_jobs=1):
        probas = self.predict_proba(X, n_jobs=n_jobs)
        predictions = np.argmax(probas, axis=1)

        return predictions
X, y = make_classification(n_samples=100000)

In [85]:
random_forest = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [86]:
%%time

_ = random_forest.fit(X, y, n_jobs=1)

CPU times: user 5 s, sys: 39.3 ms, total: 5.04 s
Wall time: 5.04 s


In [87]:
%%time

preds_1 = random_forest.predict(X, n_jobs=1)

CPU times: user 167 ms, sys: 5.4 ms, total: 172 ms
Wall time: 172 ms


In [88]:
random_forest = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [89]:
%%time

_ = random_forest.fit(X, y, n_jobs=2)

CPU times: user 27.3 ms, sys: 15.3 ms, total: 42.7 ms
Wall time: 2.78 s


In [90]:
%%time

preds_2 = random_forest.predict(X, n_jobs=2)

CPU times: user 2.47 ms, sys: 2.79 ms, total: 5.25 ms
Wall time: 3.39 ms


In [92]:
preds_1

array([1, 1, 0, ..., 1, 0, 1])

In [93]:
preds_2

array([0, 0, 0, ..., 0, 0, 0])

In [91]:
(preds_1 == preds_2).all()   # Количество worker'ов не должно влиять на предсказания

False

#### Какие есть недостатки у вашей реализации параллельного Random Forest (если они есть)? Как это можно исправить? Опишите словами, можно без кода (+1 дополнительный балл)

Ответ пишите тут

# Задание 2 (9 баллов)

Напишите декоратор `memory_limit`, который позволит ограничивать использование памяти декорируемой функцией.

Декоратор должен принимать следующие аргументы:
1. `soft_limit` - "мягкий" лимит использования памяти. При превышении функцией этого лимита должен будет отображён **warning**
2. `hard_limit` - "жёсткий" лимит использования памяти. При превышении функцией этого лимита должно будет брошено исключение, а функция должна немедленно завершить свою работу
3. `poll_interval` - интервал времени (в секундах) между проверками использования памяти

Требования:
1. Потребление функцией памяти должно отслеживаться **во время выполнения функции**, а не после её завершения
2. **warning** при превышении `soft_limit` должен отображаться один раз, даже если функция переходила через этот лимит несколько раз
3. Если задать `soft_limit` или `hard_limit` как `None`, то соответствующий лимит должен быть отключён
4. Лимиты должны передаваться и отображаться в формате `<number>X`, где `X` - символ, обозначающий порядок единицы измерения памяти ("B", "K", "M", "G", "T", ...)
5. В тексте warning'ов и исключений должен быть указан текщий объём используемой памяти и величина превышенного лимита

В задании можно использовать только модули из **стандартной библиотеки** питона, можно писать вспомогательные функции и/или классы

В коде ниже для вас предопределены некоторые полезные функции, вы можете ими пользоваться, а можете не пользоваться

In [1]:
import os
import psutil
import time
import warnings

def get_memory_usage() -> int:    # Показывает текущее потребление памяти процессом
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss


def bytes_to_human_readable(n_bytes:int) -> str:
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for idx, s in enumerate(symbols):
        prefix[s] = 1 << (idx + 1) * 10
    for s in reversed(symbols):
        if n_bytes >= prefix[s]:
            value = float(n_bytes) / prefix[s]
            return f"{value:.2f}{s}"
    return f"{n_bytes}B"

def memory_limit(soft_limit: Optional[str] = None, hard_limit: Optional[str] = None, poll_interval: float = 1) -> Callable:
    def decorator(func: Callable) -> Callable:
        def wrapper(*args, **kwargs):
            def print_warning() -> int:
                mem = get_memory_usage()
                if mem > soft_limit_bytes and not warning_printed[0]:
                    warnings.warn(f"Memory usage ({bytes_to_human_readable(mem)}) is above the soft limit ({bytes_to_human_readable(soft_limit_bytes)})")
                    warning_printed[0] = True
                return mem
            
            def check_hard_limit():
                mem = get_memory_usage()
                if mem > hard_limit_bytes:
                    raise MemoryError(f"Memory usage ({bytes_to_human_readable(mem)}) is above the hard limit ({bytes_to_human_readable(hard_limit_bytes)})")

            warning_printed = [False]   # Флаг, показывающий, было ли уже напечатано warning при превышении soft_limit
            soft_limit_bytes = None if soft_limit is None else int(soft_limit[:-1]) * (1024 ** "BKMGT".index(soft_limit[-1]))
            hard_limit_bytes = None if hard_limit is None else int(float(hard_limit[:-1]) * (1024 ** "BKMGT".index(hard_limit[-1])))
            
            t = time.time()
            max_mem = 0
            while True:
                try:
                    mem = print_warning()
                    check_hard_limit()
                    max_mem = max(max_mem, mem)
                    result = func(*args, **kwargs)
                    return result
                finally:
                    time.sleep(poll_interval)
                    if time.time() - t > 60:  # Если функция выполняется более 60 секунд, перестаём отслеживать память
                        break
            raise Exception(f"Memory usage ({bytes_to_human_readable(max_mem)}) exceeded the soft limit ({bytes_to_human_readable(soft_limit_bytes)})")
        return wrapper
    return decorator

In [4]:
@memory_limit(soft_limit="512M", hard_limit="1.5G", poll_interval=0.1)
def memory_increment():
    """
    Функция для тестирования
    
    В течение нескольких секунд достигает использования памяти 1.89G
    Потребление памяти и скорость накопления можно варьировать, изменяя код
    """
    lst = []
    for i in range(50000000):
        if i % 500000 == 0:
            time.sleep(0.1)
        lst.append(i)
    return lst


In [5]:
result = memory_increment()
print(memory_limit(result))

<function memory_limit.<locals>.decorator at 0x1055689d0>


# Задание 3 (11 баллов)

Напишите функцию `parallel_map`. Это должна быть **универсальная** функция для распараллеливания, которая эффективно работает в любых условиях.

Функция должна принимать следующие аргументы:
1. `target_func` - целевая функция (обязательный аргумент)
2. `args_container` - контейнер с позиционными аргументами для `target_func` (по-умолчанию `None` - позиционные аргументы не передаются)
3. `kwargs_container` - контейнер с именованными аргументами для `target_func` (по-умолчанию `None` - именованные аргументы не передаются)
4. `n_jobs` - количество workers, которые будут использованы для выполнения (по-умолчанию `None` - количество логических ядер CPU в системе)

Функция должна работать аналогично `***PoolExecutor.map`, применяя функцию к переданному набору аргументов, но с некоторыми дополнениями и улучшениями
    
Поскольку мы пишем **универсальную** функцию, то нам нужно будет выполнить ряд требований, чтобы она могла логично и эффективно работать в большинстве ситуаций

1. `target_func` может принимать аргументы любого вида в любом количестве
2. Любые типы данных в `args_container`, кроме `tuple`, передаются в `target_func` как единственный позиционный аргумент. `tuple` распаковываются в несколько аргументов
3. Количество элементов в `args_container` должно совпадать с количеством элементов в `kwargs_container` и наоборот, также значение одного из них или обоих может быть равно `None`, в иных случаях должна кидаться ошибка (оба аргумента переданы, но размеры не совпадают)

4. Функция должна выполнять определённое количество параллельных вызовов `target_func`, это количество зависит от числа переданных аргументов и значения `n_jobs`. Сценарии могут быть следующие
    + `args_container=None`, `kwargs_container=None`, `n_jobs=None`. В таком случае функция `target_func` выполнится параллельно столько раз, сколько на вашем устройстве логических ядер CPU
    + `args_container=None`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **5** раз
    + `args_container=[1, 2, 3]`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **3** раза, несмотря на то, что `n_jobs=5` (так как есть всего 3 набора аргументов для которых нам нужно получить результат, а лишние worker'ы создавать не имеет смысла)
    + `args_container=None`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем именованные аргументы
    + `args_container=[1, 2, 3]`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем и позиционные, и именованные аргументы
    + `args_container=[1, 2, 3, 4]`, `kwargs_container=None`, `n_jobs=2`. В таком случае в каждый момент времени параллельно будет выполняться **не более 2** функций `target_func`, так как нам нужно выполнить её 4 раза, но у нас есть только 2 worker'а.
    + В подобных случаях (из примера выше) должно оптимизироваться время выполнения. Если эти 4 вызова выполняются за 5, 1, 2 и 1 секунды, то параллельное выполнение с `n_jobs=2` должно занять **5 секунд** (не 7 и тем более не 10)

5. `parallel_map` возвращает результаты выполнения `target_func` **в том же порядке**, в котором были переданы соответствующие аргументы
6. Работает с функциями, созданными внутри других функций

Для базового решения от вас не ожидается **сверххорошая** оптимизация по времени и памяти для всех возможных случаев. Однако за хорошо оптимизированную логику работы можно получить до **+3 дополнительных баллов**

Вы можете сделать класс вместо функции, если вам удобнее

В задании можно использовать только модули из **стандартной библиотеки** питона

Ниже приведены тестовые примеры по каждому из требований

In [22]:
def parallel_map(target_func,
                 args_container=None,
                 kwargs_container=None,
                 n_jobs=None):
    # Ваш код здесь

In [57]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from itertools import repeat
import os

def parallel_map(target_func, args_container=None, kwargs_container=None, n_jobs=None):
    if args_container is None:
        args_container = [()] * len(kwargs_container)
    elif kwargs_container is None:
        kwargs_container = [{}] * len(args_container)
    
    if len(args_container) != len(kwargs_container):
        raise ValueError("args_container and kwargs_container must have the same length")
    
    if n_jobs is None:
        n_jobs = os.cpu_count()
    
    if n_jobs <= 0:
        raise ValueError("n_jobs must be a positive integer")
    
    if n_jobs > len(args_container):
        n_jobs = len(args_container)
    
    # Create an executor based on the number of jobs
    executor_type = ThreadPoolExecutor if n_jobs == 1 else ProcessPoolExecutor
    with executor_type(max_workers=n_jobs) as executor:
        # Create an iterator with the arguments to pass to target_func
        args_iterator = zip(args_container, kwargs_container)
        
        # Submit tasks to the executor and store the futures in a list
        futures = [executor.submit(target_func, *args, **kwargs) for args, kwargs in args_iterator]
        
        # Iterate over the completed futures and store the results in a list
        results = [future.result() for future in as_completed(futures)]
    
    return results

In [1]:
import multiprocessing

def parallel_map(target_func, args_container=None, kwargs_container=None, n_jobs=None):
    def wrapper(target_func, args, kwargs):
        return target_func(*args, **kwargs)

    # определяем количество заданий
    num_tasks = max([len(args_container) if args_container is not None else 0, 
                     len(kwargs_container) if kwargs_container is not None else 0])
    # определяем количество воркеров
    if n_jobs is None:
        n_jobs = multiprocessing.cpu_count()
    else:
        n_jobs = min(n_jobs, num_tasks)
    
    with multiprocessing.Pool(processes=n_jobs) as pool:
        # создаем список аргументов для функции wrapper
        args_list = []
        for i in range(num_tasks):
            args = [] if args_container is None else args_container[i] if isinstance(args_container[i], tuple) else [args_container[i]]
            kwargs = {} if kwargs_container is None else kwargs_container[i]
            args_list.append((target_func, args, kwargs))
        
        # запускаем функцию wrapper с использованием воркеров
        results = pool.starmap(wrapper, args_list)
    
    return results

In [97]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Any, List, Tuple, Union
import os

def parallel_map(target_func, args_container=None, kwargs_container=None, n_jobs=None) -> List[Any]:
    if args_container is None and kwargs_container is None:
        raise ValueError("At least one of args_container or kwargs_container must be non-None")
    
    if args_container is not None and kwargs_container is not None:
        if len(args_container) != len(kwargs_container):
            raise ValueError("args_container and kwargs_container must have same length")
    elif args_container is not None:
        kwargs_container = [{}] * len(args_container)
    else:
        args_container = [()] * len(kwargs_container)

    if n_jobs is None:
        n_jobs = os.cpu_count()
    
    if len(args_container) < n_jobs:
        n_jobs = len(args_container)
    
    results = [None] * len(args_container)

    def worker(func, args, kwargs, i):
        results[i] = func(*args, **kwargs)
    
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        futures = []
        for i, (args, kwargs) in enumerate(zip(args_container, kwargs_container)):
            futures.append(executor.submit(worker, target_func, args, kwargs, i))
        for future in as_completed(futures):
            future.result()
    
    return results

In [120]:
from concurrent.futures import ThreadPoolExecutor
from typing import Any, List, Dict, Tuple, Union
import os

def parallel_map(target_func: callable, args_container: List[Any]=None,
                 kwargs_container: List[Dict[str, Any]]=None, n_jobs: int=None) -> List[Any]:
    
    # Определяем количество задач
    if args_container is None and kwargs_container is None:
        num_tasks = 0
    elif (args_container is None and kwargs_container is not None) or (args_container is not None and kwargs_container is None):
        raise ValueError("args_container and kwargs_container must have the same length")
    else:
        num_tasks = len(args_container)
        
    # Определяем количество worker'ов
    if n_jobs is None:
        n_jobs = os.cpu_count()
    else:
        n_jobs = min(n_jobs, os.cpu_count())
    
    # Функция, которая будет вызываться в потоке
    def target_func_wrapper(*args, **kwargs):
        return target_func(*args, **kwargs)
    
    # Создаем пул потоков
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        
        # Результаты выполнения функции
        results = []
        
        # Определяем, сколько задач будем выполнять одновременно
        tasks_per_worker = max(1, num_tasks // n_jobs)
        
        # Создаем пустой список kwargs_container, если он не задан изначально
        if kwargs_container is None:
            kwargs_container = [{}] * num_tasks
        
        # Итерируемся по количеству worker'ов
        for i in range(n_jobs):
            
            # Начальный индекс задачи для текущего worker'а
            start_idx = i * tasks_per_worker
            
            # Конечный индекс задачи для текущего worker'а
            end_idx = min(num_tasks, start_idx + tasks_per_worker)
            
            # Формируем список позиционных аргументов для функции
            args = []
            if args_container is not None:
                args = args_container[start_idx:end_idx]
                # Если элементы списка args_container - кортежи, то распаковываем их
                args = [arg if isinstance(arg, tuple) else (arg,) for arg in args]
                
            # Формируем список именованных аргументов для функции
            kwargs = {}
            if kwargs_container is None:
                kwargs_container = [{}] * num_tasks
            if kwargs_container is not None:
                kwargs = kwargs_container[start_idx:end_idx]
            

                
            # Выполняем функцию в потоке
            futures = [executor.submit(target_func_wrapper, *arg, **kwarg) for arg, kwarg in zip(args, kwargs)]
            
            # Добавляем результат выполнения задач в список результатов
            for future in futures:
                results.append(future.result())
                
    return results


In [121]:
import time


# Это только один пример тестовой функции, ваша parallel_map должна уметь эффективно работать с ЛЮБЫМИ функциями
# Поэтому обязательно протестируйте код на чём-нибудбь ещё
def test_func(x=1, s=2, a=1, b=1, c=1):
    time.sleep(s)
    return a*x**2 + b*x + c

In [122]:
%%time

# Пример 2.1
# Отдельные значения в args_container передаются в качестве позиционных аргументов
parallel_map(test_func, args_container=[1, 2.0, 3j-1, 4])  # Здесь происходят параллельные вызовы: test_func(1) test_func(2.0) test_func(3j-1) test_func(4)

ValueError: args_container and kwargs_container must have the same length

In [100]:
%%time

# Пример 2.2
# Элементы типа tuple в args_container распаковываются в качестве позиционных аргументов
parallel_map(test_func, [(1, 1), (2.0, 2), (3j-1, 3), 4])    # Здесь происходят параллельные вызовы: test_func(1, 1) test_func(2.0, 2) test_func(3j-1, 3) test_func(4)

TypeError: __main__.test_func() argument after * must be an iterable, not int

In [101]:
%%time

# Пример 3.1
# Возможна одновременная передача args_container и kwargs_container, но количества элементов в них должны быть равны
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

# Здесь происходят параллельные вызовы: test_func(1, s=3) test_func(2, s=3) test_func(3, s=3) test_func(4, s=3)

TypeError: __main__.test_func() argument after * must be an iterable, not int

In [102]:
%%time

# Пример 3.2
# args_container может быть None, а kwargs_container задан явно
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

CPU times: user 1.86 ms, sys: 1.59 ms, total: 3.46 ms
Wall time: 3.01 s


[3, 3, 3, 3]

In [43]:
%%time

# Пример 3.3
# kwargs_container может быть None, а args_container задан явно
parallel_map(test_func,
             args_container=[1, 2, 3, 4])

CPU times: user 4.11 ms, sys: 9.2 ms, total: 13.3 ms
Wall time: 2.01 s


[3, 7, 13, 21]

In [44]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 500 µs, sys: 43.3 ms, total: 43.8 ms
Wall time: 2.04 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [44]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 500 µs, sys: 43.3 ms, total: 43.8 ms
Wall time: 2.04 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [32]:
%%time

# Пример 3.5
# При несовпадении количеств позиционных и именованных аргументов кидается ошибка
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}])

ValueError: Numbers of positional arguments and keyword arguments do not match: 4 and 3

In [45]:
%%time

# Пример 4.1
# Если функция не имеет обязательных аргументов и аргумент n_jobs не был передан, то она выполняется параллельно столько раз, сколько ваш CPU имеет логических ядер
# В моём случае это 24, у вас может быть больше или меньше
parallel_map(test_func)

CPU times: user 9.3 ms, sys: 51.2 ms, total: 60.5 ms
Wall time: 2.06 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [47]:
%%time

# Пример 4.2
# Если функция не имеет обязательных аргументов и передан только аргумент n_jobs, то она выполняется параллельно n_jobs раз
parallel_map(test_func, n_jobs=2)

CPU times: user 2.06 ms, sys: 5.92 ms, total: 7.99 ms
Wall time: 2.01 s


[3, 3]

In [48]:
%%time

# Пример 4.3
# Если аргументов для target_func указано МЕНЬШЕ, чем n_jobs, то используется такое же количество worker'ов, сколько было передано аргументов
parallel_map(test_func,
             args_container=[1, 2, 3],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 314 µs, sys: 8.69 ms, total: 9 ms
Wall time: 2.01 s


[3, 7, 13]

In [49]:
%%time

# Пример 4.4
# Аналогичный предыдущему случай, но с именованными аргументами
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 1.26 ms, sys: 9.47 ms, total: 10.7 ms
Wall time: 3.01 s


[3, 3, 3]

In [50]:
%%time

# Пример 4.5
# Комбинация примеров 4.3 и 4.4 (переданы и позиционные и именованные аргументы)
parallel_map(test_func,
             args_container=[1, 2, 3],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 7.88 ms, sys: 0 ns, total: 7.88 ms
Wall time: 3.01 s


[3, 7, 13]

In [50]:
%%time

# Пример 4.6
# Если аргументов для target_func указано БОЛЬШЕ, чем n_jobs, то используется n_jobs worker'ов
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=None,
             n_jobs=2)   # Здесь используется 2 worker'a

CPU times: user 7.88 ms, sys: 0 ns, total: 7.88 ms
Wall time: 3.01 s


[3, 7, 13]

In [51]:
%%time

# Пример 4.7
# Время выполнения оптимизируется, данный код должен отрабатывать за 5 секунд
parallel_map(test_func,
             kwargs_container=[{"s": 5}, {"s": 1}, {"s": 2}, {"s": 1}],
             n_jobs=2)

CPU times: user 3.03 ms, sys: 11 ms, total: 14 ms
Wall time: 5.01 s


[3, 3, 3, 3]

In [57]:
def test_func2(string, sleep_time=1):
    time.sleep(sleep_time)
    return string

# Пример 5
# Результаты возвращаются в том же порядке, в котором были переданы соответствующие аргументы вне зависимости от того, когда завершился worker
arguments = ["first", "second", "third", "fourth", "fifth"]
parallel_map(test_func2,
             args_container=arguments,
             kwargs_container=[{"sleep_time": 5}, {"sleep_time": 4}, {"sleep_time": 3}, {"sleep_time": 2}, {"sleep_time": 1}])

['first', 'second', 'third', 'fourth', 'fifth']

In [58]:
%%time


def test_func3():
    def inner_test_func(sleep_time):
        time.sleep(sleep_time)
    return parallel_map(inner_test_func, args_container=[1, 2, 3])

# Пример 6
# Работает с функциями, созданными внутри других функций
test_func3()

[None, None, None]